In [1]:
!pip install -q tensorflow-recommenders
!pip install -q scann

You should consider upgrading via the 'C:\Users\admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement scann (from versions: none)
ERROR: No matching distribution found for scann
You should consider upgrading via the 'C:\Users\admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import tensorflow as tf
import seaborn as sns
import math
import sys
from math import sqrt


from tensorflow.keras.layers import *
from keras import backend as K
from keras.layers import Embedding, Flatten, Concatenate
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input, Reshape, Dot
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import roc_curve, accuracy_score
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import mean_squared_error, r2_score

import pprint
import tempfile
from typing import Dict, Text

import tensorflow_recommenders as tfrs

# set the max columns to none
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')
%matplotlib inline

In [245]:
import logging
tf.get_logger().setLevel(logging.ERROR)

In [73]:
train_hotel = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/Train%20and%20Test%20Data/Train_Hotel.csv', index_col=[0])
train_user = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/Train%20and%20Test%20Data/Train_User.csv', index_col=[0])

test_hotel = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/Train%20and%20Test%20Data/Test_Hotel.csv', index_col=[0])
test_user = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/Train%20and%20Test%20Data/Test_User.csv', index_col=[0])

In [342]:
df_hotel_ML = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/csv_final/Final_Dataset_Hotel_ML.csv', index_col=[0])
df_user_ML = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/csv_final/Final_Dataset_User_ML.csv', index_col=[0])
df_hotel_nonML = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/csv_final/Final_Dataset_Hotel_nonML.csv', index_col=[0])
df_user_nonML = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/csv_final/Final_Dataset_User_nonML.csv', index_col=[0])

In [75]:
# mengubah hotel_id ke integer
for i,item in train_hotel.iterrows():
    train_hotel.loc[i, 'hotel_id'] = int(train_hotel.loc[i, 'hotel_id'][1:])
train_hotel[["hotel_id"]] = train_hotel[["hotel_id"]].apply(pd.to_numeric)

In [76]:
unique_hotel_ids = df_hotel_ML['hotel_id'].unique()
unique_user_ids = df_user_ML['user_id'].unique()
print(len(unique_hotel_ids))
print(len(unique_user_ids))

1311
9219


In [206]:
df_user_ML

,user_id,most_reviewed,reviewed_hotel,latitude,longitude,hotel_star_rating,site_review_rating,Beach Hut,BnB,Bungalow,Cottage,Farm Stay,Guest House,Homestay,Hostel,Hotel,Houseboat,Lodge,Motel,Palace,Resort,Service Apartment,Tent,Villa,Bike on Rent,Drivers Rest Room,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Amphitheatre,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Bowling,Business Center,CCTV,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Courier Service,Currency Exchange,Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,In House Events,Internet Access - Surcharge,Jacuzzi,Karaoke,Kids Play Zone,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Major Credit Cards Accepted,Marina On Site,Massage Services,Medical Assistance Available,Metal Detectors,Mini Theatre,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Night Club,Open Air Restaurant / Dining,Oxygen Cylinder (on request),Paid Transfers,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Table,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Skiing Facility Available,Smoke Detector,Smoking Area,Solarium,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Valet service,Vegetarian Food / Jain Food Available,Veranda,Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,120 AC,220 AC,Air Conditioning,Air Cooler,Alarm Clock,Attached Bathroom,Balcony,Bathrobe,Bathrobes,Bathroom Amenities,Bathtub,Bathtub Only,Bedside Table,Bidet,Blackout Drapes,Breakfast Available,Breakfast chargeable,Ceiling Fan,Centrally Heated,Channel Music,Climate Control,Closet,Clothes Dryer,Clothes Hook in Bathroom,Coffee Table,Complete Bedding,Complimentary Use of Bicycle,Computer,Cribs Available,Crockery / Cutlery,Curtains/Shades,Daily Room Cleaning,Desk,Direct Dial Phone,Disabled Friendly Bathrooms,Do-Not-Disturb Notice,Dressing Area,Dustbins,Ensuite / Private Bathroom,"Extra Bed, Towels, Linens, Bedding (on request)",Extra Toilteries on Demand,Full Kitchen,Gas Cylinder - Chargeable,HD / LCD Satellite TV,Hairdryer,Handicapped Facilities,Hill View from all Rooms,Hot / Cold Running Water,Housekeeping,Ice Bucket on Demand,In-room Storage Space,Induction Plate,International Switch Plugs (on request),Kitchenette,Luggage Rack,Mineral Water,Mini Bar,Mini Refrigerator,Mirror,Modern Toilet Facilities,Movie Channels,Movies &amp; Games on demand,Newspapers,No Smoking Room,Patio - Property,Phone Messaging Services,Pillow menu,Premium Amenities / Toiletries,Premium Beddings / Hypo-Allergenic,Primium Bathrooms,Private Garden,Radio,Reading Lamps,Remote Control TV

In [207]:
# define tf dataset for hotel and ratings
tf_data_hotel = tf.data.Dataset.from_tensor_slices(dict(df_hotel_ML))
tf_data_user = tf.data.Dataset.from_tensor_slices(dict(df_user_ML))

ratings = tf_data_user.map(lambda x: {
    "hotel_id": x["most_reviewed"],
    "user_id": x["user_id"],
    "user_rating": x["site_review_rating"]
})
hotels = tf_data_hotel.map(lambda x: x["hotel_id"])

In [209]:
# get hotel id and user id from ratings
hotel_ids = ratings.batch(1_000_000).map(lambda x: x['hotel_id'])
user_ids = ratings.batch(1_000_000).map(lambda x: x['user_id'])

In [228]:
# set user data train and test
tf_train_user = tf.data.Dataset.from_tensor_slices(dict(train_user))
train_ratings = tf_train_user.map(lambda x: {
    "hotel_id": x["most_reviewed"],
    "user_id": x["user_id"],
    "user_rating": x["site_review_rating"]
})

tf_test_user = tf.data.Dataset.from_tensor_slices(dict(test_user))
test_ratings = tf_test_user.map(lambda x: {
    "hotel_id": x["most_reviewed"],
    "user_id": x["user_id"],
    "user_rating": x["site_review_rating"]
})

In [101]:
train_ratings

<MapDataset element_spec={'hotel_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None)}>

### Buat Query Tower 

In [11]:
embedding_dimension = 32

In [12]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

### Buat Candidate Tower 

In [20]:
hotel_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_hotel_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_hotel_ids) + 1, embedding_dimension)
])

### Metrics 

In [103]:
metrics = tfrs.metrics.FactorizedTopK(candidates=hotels.batch(128).map(hotel_model))

### Loss 

In [104]:
task = tfrs.tasks.Retrieval(metrics=metrics)

# Model 

### Base Retrieval Model 

In [197]:
class HotelPrediction(tfrs.Model):

  def __init__(self, user_model, hotel_model):
    super().__init__()
    self.hotel_model: tf.keras.Model = hotel_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.hotel_model(features["hotel_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [198]:
model = HotelPrediction(user_model, hotel_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

### Ranking Model with Hotel Ratings

In [374]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32
    
    # hitung embeddings untuk user
    self.user_embeddings = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])
    
    # hitung embeddings untuk hotel
    self.hotel_embeddings = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_hotel_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_hotel_ids) + 1, embedding_dimension)
    ])

    # hitung rating
    self.ratings = tf.keras.Sequential([
        # mempelajari beberapa dense layer
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        # membuat prediksi di layer final
        tf.keras.layers.Dense(1),
    ])
    
  def call(self, inputs):
    
    user_id, hotel_id = inputs
    
    user_embedding = self.user_embeddings(user_id)
    hotel_embedding = self.hotel_embeddings(hotel_id)
    
    return self.ratings(tf.concat([user_embedding,hotel_embedding], axis=1))
    

In [375]:
class HotelPrediction2(tfrs.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    rating_predictions = self.ranking_model(
        (features['user_id'], features['hotel_id']))

    # The task computes the loss and the metrics.
    return self.task(labels=features['user_rating'], predictions=rating_predictions)

In [376]:
model = HotelPrediction2()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [377]:
cached_train = train_ratings.shuffle(100_000).batch(8192).cache()
cached_test = test_ratings.batch(4096).cache()

In [378]:
model.fit(cached_train, epochs=10)

Epoch 1/10
1/1 [==============================] - 1s 808ms/step - root_mean_squared_error: 4.1185 - loss: 16.9621 - regularization_loss: 0.0000e+00 - total_loss: 16.9621
Epoch 2/10
1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 2.7318 - loss: 7.4628 - regularization_loss: 0.0000e+00 - total_loss: 7.4628
Epoch 3/10
1/1 [==============================] - 0s 18ms/step - root_mean_squared_error: 3.2747 - loss: 10.7236 - regularization_loss: 0.0000e+00 - total_loss: 10.7236
Epoch 4/10
1/1 [==============================] - 0s 15ms/step - root_mean_squared_error: 3.8818 - loss: 15.0684 - regularization_loss: 0.0000e+00 - total_loss: 15.0684
Epoch 5/10
1/1 [==============================] - 0s 15ms/step - root_mean_squared_error: 3.2175 - loss: 10.3525 - regularization_loss: 0.0000e+00 - total_loss: 10.3525
Epoch 6/10
1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 2.5458 - loss: 6.4809 - regularization_loss: 0.0000e+00 - total_lo

In [379]:
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 0s 336ms/step - root_mean_squared_error: 0.9814 - loss: 0.9632 - regularization_loss: 0.0000e+00 - total_loss: 0.9632


{'root_mean_squared_error': 0.9814107418060303,
 'loss': 0.9631670713424683,
 'regularization_loss': 0,
 'total_loss': 0.9631670713424683}

In [224]:
# !mkdir -p saved_model
model.save('saved_model/model1')

ValueError: Model <__main__.HotelPrediction2 object at 0x000001BEC81A7EE0> cannot be saved either because the input shape is not available or because the forward pass of the model is not defined.To define a forward pass, please override `Model.call()`. To specify an input shape, either call `build(input_shape)` directly, or call the model on actual data using `Model()`, `Model.fit()`, or `Model.predict()`. If you have a custom training step, please make sure to invoke the forward pass in train step through `Model.__call__`, i.e. `model(inputs)`, as opposed to `model.call()`.

# Evaluation

### Making Predictions Model 1

In [174]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((hotels.batch(100), hotels.batch(100).map(model.hotel_model)))
)

# Get recommendations.
user_pred = "U000001"
_, titles = index(tf.constant([user_pred]))
print(f"Recommendations for user {user_pred}: {titles[0, :]}")

Recommendations for user U000001: [b'H000021' b'H000023' b'H000031' b'H000080' b'H000596' b'H001281'
 b'H000768' b'H000180' b'H000543' b'H000882']


### Making Predictions Model 2

In [382]:
for m in test_ratings.take(5):
    print(m['hotel_id'].numpy())

b'H000763'
b'H000906'
b'H001153'
b'H000239'
b'H000236'


In [408]:
dict_test_ratings = {}
user_id = "U000001"
user_pred = tf.constant([user_id])

for m in test_ratings.take(10):
    dict_test_ratings[m['hotel_id'].numpy()] = RankingModel()((user_pred,[m['hotel_id']]))
   
for m in sorted(dict_test_ratings, key=dict_test_ratings.get, reverse=True):
    print(m)


b'H000290'
b'H000763'
b'H000748'
b'H000239'
b'H001153'
b'H000906'
b'H000236'
b'H001283'
b'H000091'
b'H000088'


In [409]:
dict_test_ratings

{b'H000763': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.01983745]], dtype=float32)>,
 b'H000906': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00133869]], dtype=float32)>,
 b'H001153': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00119385]], dtype=float32)>,
 b'H000239': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.00019254]], dtype=float32)>,
 b'H000236': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00612736]], dtype=float32)>,
 b'H001283': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.01294306]], dtype=float32)>,
 b'H000748': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.01362267]], dtype=float32)>,
 b'H000091': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.02478851]], dtype=float32)>,
 b'H000290': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.02427823]], dtype=float32)>,
 b'H000088': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.02787884]], dtype=float32)>}

### See Predictions Result

In [236]:
arr_result = []
for item in titles.numpy()[0]:
    arr_result.append(item.decode("utf-8"))
arr_result

arr_result = ['H000763',
 'H000906',
 'H001153',
 'H000236',
 'H000239',]

In [238]:
df_hotel_ML[df_hotel_ML['hotel_id'].isin(arr_result)]

,hotel_id,total_review,latitude,longitude,hotel_star_rating,site_review_rating,Beach Hut,BnB,Bungalow,Cottage,Farm Stay,Guest House,Homestay,Hostel,Hotel,Houseboat,Lodge,Motel,Palace,Resort,Service Apartment,Tent,Villa,Bike on Rent,Drivers Rest Room,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Amphitheatre,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Bowling,Business Center,CCTV,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Courier Service,Currency Exchange,Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,In House Events,Internet Access - Surcharge,Jacuzzi,Karaoke,Kids Play Zone,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Major Credit Cards Accepted,Marina On Site,Massage Services,Medical Assistance Available,Metal Detectors,Mini Theatre,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Night Club,Open Air Restaurant / Dining,Oxygen Cylinder (on request),Paid Transfers,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Table,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Skiing Facility Available,Smoke Detector,Smoking Area,Solarium,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Valet service,Vegetarian Food / Jain Food Available,Veranda,Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,120 AC,220 AC,Air Conditioning,Air Cooler,Alarm Clock,Attached Bathroom,Balcony,Bathrobe,Bathrobes,Bathroom Amenities,Bathtub,Bathtub Only,Bedside Table,Bidet,Blackout Drapes,Breakfast Available,Breakfast chargeable,Ceiling Fan,Centrally Heated,Channel Music,Climate Control,Closet,Clothes Dryer,Clothes Hook in Bathroom,Coffee Table,Complete Bedding,Complimentary Use of Bicycle,Computer,Cribs Available,Crockery / Cutlery,Curtains/Shades,Daily Room Cleaning,Desk,Direct Dial Phone,Disabled Friendly Bathrooms,Do-Not-Disturb Notice,Dressing Area,Dustbins,Ensuite / Private Bathroom,"Extra Bed, Towels, Linens, Bedding (on request)",Extra Toilteries on Demand,Full Kitchen,Gas Cylinder - Chargeable,HD / LCD Satellite TV,Hairdryer,Handicapped Facilities,Hill View from all Rooms,Hot / Cold Running Water,Housekeeping,Ice Bucket on Demand,In-room Storage Space,Induction Plate,International Switch Plugs (on request),Kitchenette,Luggage Rack,Mineral Water,Mini Bar,Mini Refrigerator,Mirror,Modern Toilet Facilities,Movie Channels,Movies &amp; Games on demand,Newspapers,No Smoking Room,Patio - Property,Phone Messaging Services,Pillow menu,Premium Amenities / Toiletries,Premium Beddings / Hypo-Allergenic,Primium Bathrooms,Private Garden,Radio,Reading Lamps,Remote Control TV,Remote Control

In [349]:
pd.DataFrame(df_hotel_nonML.iloc[20]).T

,hotel_id,hotel_name,hotel_description,address,point_of_interest,city,state,country,latitude,longitude,property_type,hotel_star_rating,site_review_rating,hotel_facilities,room_facilities,hotel_images,hotel_price
20,H000021,Best Western Plus South Coast Inn,The standard check-in time is 12:00 PM and the...,5620 Calle Real,['Mallandur'],Goleta,CA,US,34.4,-119.8,Homestay,1,1.0,"('Doctor on Call', 'Gardens', 'Guide / Sightse...","('Attached Bathroom', 'Balcony', 'Bathroom Ame...",https://raw.githubusercontent.com/zulfauzi92/H...,270000.0


In [348]:
pd.DataFrame(df_hotel_nonML.iloc[302]).T

,hotel_id,hotel_name,hotel_description,address,point_of_interest,city,state,country,latitude,longitude,property_type,hotel_star_rating,site_review_rating,hotel_facilities,room_facilities,hotel_images,hotel_price
302,H000303,The St. Regis San Francisco,The standard check-in time is 12:00 PM and the...,125 3rd Street,['Others'],San Francisco,CA,US,37.8,-122.4,Hotel,1,2.0,"('Doctor on Call',)","('Attached Bathroom',)",https://raw.githubusercontent.com/zulfauzi92/H...,225000.0


In [358]:
pd.DataFrame(df_hotel_nonML.iloc[978]).T

,hotel_id,hotel_name,hotel_description,address,point_of_interest,city,state,country,latitude,longitude,property_type,hotel_star_rating,site_review_rating,hotel_facilities,room_facilities,hotel_images,hotel_price
978,H000979,Best Western Plus Grand Island Inn Suites,The standard check-in time is 12:00 PM and the...,2707 S Locust St,['Kaimara'],Grand Island,NE,US,40.9,-98.3,Homestay,0,5.0,"('Campfire / Bon Fire', 'Games Zone', 'Gardens...","('Bathroom Amenities', 'Desk', 'Hot / Cold Run...",https://raw.githubusercontent.com/zulfauzi92/H...,260000.0


In [191]:
arr_prediction = []
for item in arr_result:
    arr_prediction.append(int(item[1:]))
arr_prediction

[21, 23, 31, 80, 596, 1281, 768, 180, 543, 882]